# Reflexions sur le débruitage

## SOMMAIRE

#### I. Types de bruit

* Bruit Gaussien
* Bruit de Poisson
* Bruit impulsionnel
* Bruit uniforme

#### II. Techniques de débruitage

* **A. Filtres**
  * Filtre moyen
  * Filtre médian
  * Filtre gaussien

* **B. Statistiques**
  * PCA
    * Version de `sklearn`
    * _Ma version_
  * Non Local Means
    * Version de `opencv`
    * _Ma version_

* **C. Réseaux de neurones**
  * Auto encodeurs
  * CNNs

* **D. Débruiter des images plus complexes (EXR, 32 Bits Float...)**

___

### Introduction

Le débruitage d'images est une technique utilisée en traitement d'images pour réduire ou éliminer le bruit présent dans une image. Le bruit est une perturbation indésirable généralement introduite lors de l'acquisition de l'image par des capteurs électroniques.

Le débruitage d'images est crucial dans de nombreux domaines tels que :

* **Imagerie médicale** : Pour améliorer la clarté des images obtenues par radiographie, IRM, etc.
* **Photographie numérique** : Pour améliorer la qualité des photos prises dans des conditions de faible luminosité.
* **Surveillance et sécurité** : Pour améliorer les images des caméras de surveillance.
* **Astronomie** : Pour éliminer le bruit des images capturées par des télescopes.

### I. Types de Bruit

#### Bruit Gaussien

Bruit aléatoire suivant une distribution normale, souvent dû à des perturbations thermiques dans les capteurs.

#### Bruit de Poisson

Bruit dépendant de l'intensité du signal, typique dans les images acquises avec peu de lumière.

#### Bruit Impulsionnel (Sel et Poivre)

Apparition de pixels noirs et blancs aléatoirement répartis, souvent causé par des erreurs de transmission.

#### Bruit Uniforme

Bruit avec une distribution de probabilité uniforme sur une plage spécifique.

In [2]:
import time
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter, median_filter
import plotly.express as px
from PIL import Image
import cv2
from skimage import img_as_float
import matplotlib.pyplot as plt
import OpenEXR
import Imath
import scipy.ndimage
import cv2
from skimage.restoration import denoise_nl_means

### II. Techniques de Débruitage

Les méthodes de débruitage peuvent être classées en plusieurs catégories principales :

#### A. Filtres

In [3]:
X = pd.read_csv("src/noisy_images.csv")
X.shape

(20, 784)

In [4]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,-176.321245,-52.743850,143.604939,-53.567749,80.289381,20.686379,-197.423519,-229.401206,-221.900009,64.194187,...,34.407070,1.950735,-25.095565,133.684095,-21.664094,-94.305438,-55.987821,-89.929231,40.394774,-214.754480
1,-158.421239,16.371695,62.810879,263.533916,-193.920320,-25.366668,107.062706,125.403427,83.536343,-55.713640,...,-5.419961,-123.584030,20.240434,-25.699206,-128.545930,52.525885,-54.214887,-133.842624,-30.141215,210.408665
2,-290.343750,81.586500,12.615232,146.567851,111.233602,-188.989259,-101.464605,-107.015195,-13.069827,-245.921093,...,-139.909318,-85.214133,167.495617,62.402411,-144.402970,152.263950,-4.687051,-59.270131,-93.193600,188.229794
3,-208.840590,136.190431,38.552191,-67.825346,24.316303,176.103673,31.581298,-163.582673,29.777077,-110.969396,...,-131.385192,40.329733,-10.111639,163.497435,41.010287,-21.408008,328.274235,-15.341672,121.570863,151.757537
4,-328.876288,-42.862900,174.651874,-228.833439,71.909654,-97.206392,48.048853,-34.071313,3.820465,137.807730,...,-135.713199,-71.396796,155.237981,-141.860908,155.657335,166.609760,-52.911774,267.150703,-36.749672,131.913772


In [5]:
def apply_filters(images, side_length: int, sigma: int = None, size: int = None):
    rows = []
    for image in images:
        image = image.reshape(side_length, side_length)
        if size:
            image = median_filter(image, size=size)
        if sigma:
            image = gaussian_filter(image, sigma=sigma)
        rows.append(image.flatten())

    return pd.DataFrame(rows)

##### Filtre Moyen

Chaque pixel est remplacé par la moyenne des pixels de son voisinage.

##### Filtre Median

Chaque pixel est remplacé par la médiane des pixels de son voisinage, efficace contre le bruit impulsionnel.

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/median_low.png)
!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/median_high.png)

##### Filtre Gaussien

Utilise une convolution avec une fonction gaussienne, réduisant le bruit gaussien de manière plus lisse.

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/gaussian_low.png)
!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/gaussian_high.png)

#### B. Statistiques

##### PCA

In [6]:
class MyPCA:
    def __init__(self, n_components: int = None) -> None:
        self._components = n_components

    def normalize_x(self, X):
        mean = np.mean(X, axis=0)
        std_dev = np.std(X, axis=0)

        X = (X - mean) / std_dev

        return X, mean, std_dev

    def fit_transform(self, X):
        if self._components is None:
            self._components = X.shape[1]

        # 1. Normalize X
        X, self._X_mean, self._X_std_dev = self.normalize_x(X)

        # 2. Calculate covariance matrix
        cov_matrix = np.cov(X, rowvar=False)

        # 3. Calculate eigen-vectors and eigen-values
        self._eig_vals, self._eig_vecs = np.linalg.eig(cov_matrix)

        # 4. Deduct PCA
        indices = np.argsort(self._eig_vals)[::-1]

        sort_eig_vecs = self._eig_vecs[:, indices]
        self._sort_eig_vals = self._eig_vals[indices]

        self._sel_eig_vecs = sort_eig_vecs[:, :self._components]

        return np.dot(X, self._sel_eig_vecs)

    def explained_variance_ratio_(self):
        return np.real(self._sort_eig_vals / np.sum(self._sort_eig_vals))[:self._components]

    def inverse_transform(self, X_pca):
        dot_product = np.dot(X_pca, self._sel_eig_vecs.T)
        return np.real((dot_product * self._X_std_dev) + self._X_mean)  # avoid complex issues

In [7]:
def denoise(images: pd.DataFrame,
            pca_components: float = None,
            gaussian_strength: int = None,
            median_strength: int = None):

    side_length = int(np.sqrt(images.shape[1]))

    my_pca = MyPCA(n_components=pca_components)
    images = my_pca.fit_transform(images.values)
    images = my_pca.inverse_transform(images)

    images = apply_filters(images,
                           side_length,
                           sigma=gaussian_strength,
                           size=median_strength)

    return pd.DataFrame(images), my_pca

SKLearn PCA :
!["skpca"](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/PCA_sklearn.png)

Manual PCA
!["mypca"](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/PCA_custom.png)

In [ ]:
print(f"Manual PCA -> {pca.explained_variance_ratio_().sum()} in {period} seconds")


* **SKLearn PCA :** 84.999 % in 0.119 seconds
* **My version of PCA :** -> 84.0344 % in 0.839 seconds

##### Testing on "lena_bruit.png" (RGB, 512*512px)

In [ ]:
# Split image

def split_into_blocks(channel, block_size=70):
    h, w = channel.shape
    # Ensure the dimensions are multiples of block_size
    h_new = (h // block_size) * block_size
    w_new = (w // block_size) * block_size
    channel = channel[:h_new, :w_new]
    
    blocks = (channel.reshape(h_new // block_size, block_size, -1, block_size)
              .swapaxes(1, 2)
              .reshape(-1, block_size, block_size))
    return blocks


def blocks_to_dataframe(blocks):
    num_blocks = blocks.shape[0]
    df = pd.DataFrame(blocks.reshape(num_blocks, -1))
    return df


# Load the image
image_path = "src/lena_bruit.png"
image = Image.open(image_path)

image_np = np.array(image)

# Separate the color channels
red_channel = image_np[:, :, 0]
green_channel = image_np[:, :, 1]
blue_channel = image_np[:, :, 2]

red_blocks = split_into_blocks(red_channel)
green_blocks = split_into_blocks(green_channel)
blue_blocks = split_into_blocks(blue_channel)

blocks_to_dataframe(red_blocks).to_csv("src/R.csv", index=False)
blocks_to_dataframe(green_blocks).to_csv("src/G.csv", index=False)
blocks_to_dataframe(blue_blocks).to_csv("src/B.csv", index=False)

In [ ]:
# Denoise image

X = pd.read_csv("src/R.csv")
Y = pd.read_csv("src/G.csv")
Z = pd.read_csv("src/B.csv")

for layer, df in zip(("R", "G", "B"), (X, Y, Z)):
    result, pca = denoise(df, pca_components=50, gaussian_strength=.6, median_strength=3)
    result.to_csv(f"output/denoised_{layer}.csv", index=False)

In [ ]:
# Reconstruct image

def reconstruct_image_from_blocks(df, h_blocks, w_blocks, block_size=5):
    blocks = df.values.reshape((h_blocks, w_blocks, block_size, block_size))
    blocks = blocks.swapaxes(1, 2).reshape(h_blocks * block_size, w_blocks * block_size)
    return blocks


df_red = pd.read_csv("output/denoised_R.csv")
df_green = pd.read_csv("output/denoised_G.csv")
df_blue = pd.read_csv("output/denoised_B.csv")

block_size = 70
h_blocks = int(np.sqrt(len(df_red)))
w_blocks = h_blocks

red_channel_reconstructed = reconstruct_image_from_blocks(df_red, h_blocks, w_blocks, block_size)
green_channel_reconstructed = reconstruct_image_from_blocks(df_green, h_blocks, w_blocks, block_size)
blue_channel_reconstructed = reconstruct_image_from_blocks(df_blue, h_blocks, w_blocks, block_size)

reconstructed_image_np = np.stack((red_channel_reconstructed, green_channel_reconstructed, blue_channel_reconstructed), axis=-1)
reconstructed_image = Image.fromarray(reconstructed_image_np.astype('uint8'))


reconstructed_image.save("output/lena_bruit_pca_100_gauss_06_median_3_size_70.png")
reconstructed_image.show()

From this :

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/src/base_lena_bruit.png)


To this (my favourites are the green ones):
!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/patch_denoise_pca_median_gauss.jpg)

##### Non Local Means

Moyennage des pixels ayant des patchs similaires dans l'image, prenant en compte des pixels éloignés pour un meilleur lissage.
I was a little frustrated.
Despite the rather promising results, the quality of RGB denoising isn't really up to scratch with what we tested.

So I went looking for the best known software methods and discovered the **non-local means** method.

1. For each pixel, the NLM compares it with other pixels in the image, more or less distant.
2. A weight is then calculated based on the similarity between the patches centered around the target pixel and the comparison pixel. The more similar the patches, the higher the weight.
3. The target pixel is then replaced by a weighted average of all comparison pixels, using the calculated weights. This means that similar pixels have more influence on the final pixel value.

**Result : Detail is preserved!**

`open-cv` does this in one line and very quickly, but the aim of this project is to understand how it works. So I've redone my own version (which works even though it's _40 times_ slower!).

###### NLM openCV

In [ ]:
def compare_results(A, B):
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(A)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title(f"Result")
    plt.imshow(B)
    plt.axis("off")

    plt.show()

In [ ]:
image = cv2.imread("src/base_lena_bruit.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR to RGB

image_denoised = cv2.fastNlMeansDenoisingColored(image_rgb, None, 25, 25, 7, 21)

compare_results(image_rgb, image_denoised)

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/NLM_opencv.png)

###### NLM custom

In [8]:
def custom_NlMeansDenoisingColored(image: np.array,
                                   patch_size: int = 3,
                                   search_window_size: int = 21,
                                   h: int = 10):
    """Denoise RGB image with non local means

    Args:
        image (np.array): RGB image
        patch_size (int, optional): Size of sub-images to compare with others. Defaults to 3 -> matrix 3*3
        search_window_size (int, optional): search area. Defaults to 21.
        h (int, optional): intensity of the filter. Low = keep details, High = no details
    """
    image = img_as_float(image)  # float pour des calculs plus precis

    denoised = np.zeros_like(image)  # new image, same shape but only 0
    
    pad_size = search_window_size // 2  # fenetre de recherche
    patch_radius = patch_size // 2
    padded_image = np.pad(image,
                          ((pad_size, pad_size),
                           (pad_size, pad_size),
                           (0, 0)),
                           "reflect")
    
    # gauss
    gaussian_kernel = np.exp(
        -0.5
        * (np.linspace(-patch_radius, patch_radius, patch_size) ** 2)
        / (h ** 2))
    gaussian_kernel = gaussian_kernel[:, None] * gaussian_kernel[None, :]
    
    # patches extraction
    patches = np.lib.stride_tricks.as_strided(
        padded_image,
        shape=(padded_image.shape[0] - patch_size + 1,
               padded_image.shape[1] - patch_size + 1,
               patch_size,
               patch_size,
               image.shape[2]),
        strides=padded_image.strides[:2] + padded_image.strides[:2] + padded_image.strides[2:]
    )
    
    # original image, loop sur chaque pixel
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # coords du pixel au centre des recherches
            i1 = i + pad_size
            j1 = j + pad_size
            
            # define search limits, borders
            ref_patch = padded_image[i1-patch_radius:i1+patch_radius+1,
                                     j1-patch_radius:j1+patch_radius+1, :]

            i_min = max(i1 - pad_size, 0)
            i_max = min(i1 + pad_size + 1, padded_image.shape[0] - patch_size + 1)
            j_min = max(j1 - pad_size, 0)
            j_max = min(j1 + pad_size + 1, padded_image.shape[1] - patch_size + 1)
            
            # patches extraction
            search_window = patches[i_min:i_max, j_min:j_max]
            search_window = search_window.reshape(-1,
                                                  patch_size,
                                                  patch_size,
                                                  image.shape[2])
            
            # distance between ref patch and patch
            distances = np.sum(
                (search_window - ref_patch[None, :, :, :]) ** 2
                * gaussian_kernel[None, :, :, None],
                axis=(1, 2, 3))
            
            # weight by distances
            weights = np.exp(-distances / (h ** 2))
            weights /= np.sum(weights)
            
            # result
            denoised[i, j, :] = np.sum(weights[:, None] * search_window[:, patch_radius, patch_radius, :], axis=0)
    
    return denoised

In [ ]:
image = cv2.imread("src/lena_bruit.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_denoised = custom_NlMeansDenoisingColored(image_rgb, patch_size=5, search_window_size=15, h=1)

compare_results(image_rgb, image_denoised)

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/NLM_custom.png)

#### C. Réseaux de neurones

##### Autoencodeurs

Réseaux de neurones entraînés pour encoder puis décoder une image, apprenant à éliminer le bruit lors de la reconstruction.

##### CNNs (Convolutional Neural Networks)

Réseaux convolutifs spécialement conçus pour le débruitage d'images, exploitant des architectures profondes pour améliorer la qualité de l'image débruitée.

#### D. Débruiter des images plus complexes (EXR, 32 bits Float...)

In [ ]:
def read_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    channels = {}
    for channel_name in exr_file.header()['channels'].keys():
        channel_data = exr_file.channel(channel_name, Imath.PixelType(Imath.PixelType.FLOAT))
        channel = np.frombuffer(channel_data, dtype=np.float32).reshape(size[1], size[0])
        channels[channel_name] = channel
    
    return channels, size

def write_exr(file_path, channels, size):
    header = OpenEXR.Header(size[0], size[1])
    for channel_name in channels.keys():
        header['channels'][channel_name] = Imath.Channel(Imath.PixelType(Imath.PixelType.FLOAT))
    
    exr_file = OpenEXR.OutputFile(file_path, header)
    channel_data = {name: data.tobytes() for name, data in channels.items()}
    exr_file.writePixels(channel_data)

patch_kw = dict(patch_size=5, patch_distance=6, channel_axis=-1)  # 5x5 patches  # 13x13 search area
def denoise_image(channels):
    denoised_channels = {}
    for channel_name, channel in channels.items():
        denoised_channel = scipy.ndimage.median_filter(channel, size=12)
        denoised_channel = scipy.ndimage.gaussian_filter(channel, sigma=1)
        denoised_channels[channel_name] = denoised_channel
    return denoised_channels

def main(input_file, output_file):
    channels, size = read_exr(input_file)
    denoised_channels = denoise_image(channels)
    write_exr(output_file, denoised_channels, size)

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/src/base_rafale.jpg)

!["."](https://raw.githubusercontent.com/tristanGIANDO/gt_denoiser/main/output/images/rafale.jpg)
